In [5]:
!pip install feedparser # RSS에서 xml태그별 정보추출(예: title, link, ..)
!pip install newspaper3k # 인터넷 신문기사 분석(아래 'Article()'를 사용하기 위함)
!pip install konlpy # 한국어 형태소 분석기(주어진 문장에서 명사만 추출)

import feedparser # !pip install feedparser
from newspaper import Article # !pip install newspaper3k
from konlpy.tag import Okt # !pip install konlpy
from collections import Counter # 명사추출후에 본문에 몇번나오는지 확인(TF 계산용)
from operator import eq # 동일한 단어인지 여부를 판단
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html태그 삭제

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=8852f261a887d1186adbcc91641de04ee87d61850d1e02ebffcac5a7add9aafd
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=b1d71c504e2b1269427b0a7ad3dfe2f1d8862b3ca8ba8b864f1c731e0afe2b47
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58d

In [14]:
# 단계1: 모든 RSS파일(xml형식)을 돌아다니며 기사의 제목/link를 추출함
# urls 는 우리가 검색할 RSS의 목록을 list로 만든 것
urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml",
        ]

# 아래 함수는 RSS목록의 list 안에 존재하는 모든 기사의 title, link를 list로 구성
def crawl_rss(urls):
    array_rss = [] # 함수 시작하는 시점에 빈 리스트를 만듦. 여기에 모든 기사 채울것임
    titles_rss = set() # 중복기사제거: 기사제목들의 집합을 구성(집합은 중복 불허)
    for url in urls: # 4개의 RSS파일을 하나씩 방문(4번 돌것임)
        print("[Crawl RSS]",url) # 현재 어디에 있는지 출력
        parse_rss = feedparser.parse(url) # 현재 url를 파싱해서 그 결과를 parse_rss에 저장
        for p in parse_rss.entries:  # parse_rss에 있는 모든 entires/기사를 검색
            if p.title not in titles_rss: # 중복기사제거 : 만약 titles_rss에 동일한 제목이 없다면 추가
                array_rss.append({'title':p.title, 'link':p.link}) # 기사에서 제목/link 추출후 리스트에 추가
                titles_rss.add(p.title) # 중복기사제거 : 집합에 현재 기사제목이 없을 때만 추가
            else:
                print('Duplicated Title:',p.title) # 중복기사제거 : 중복되는 기사의 제목출력
    return array_rss

list_articles = crawl_rss(urls)
print(list_articles)

[Crawl RSS] http://rss.etnews.com/Section901.xml
[Crawl RSS] http://rss.etnews.com/Section902.xml
Duplicated Title: 폴스타, 폴스타3 사막 주행 테스트 완료…내년 1분기 韓 출시
Duplicated Title: 우크라가 폭사했다던 러 해군 사령관…다음날 화상회의 등장
Duplicated Title: 두께 7.6mm...모토로라, 5G 스마트폰 '엣지 40' 국내 출시
[Crawl RSS] http://rss.etnews.com/Section903.xml
Duplicated Title: 아이폰 'No'... 미군이 갤럭시 S23을 왜?
Duplicated Title: [단독]애플, 韓 카드사에 애플페이 계약조건 공개한다
Duplicated Title: 문턱 높은 E-7 비자 요건, SW 인력난 가중
Duplicated Title: SKT "AI컴퍼니 도약…매출 25兆 달성"
Duplicated Title: 이재명, 구속영장 기각… 법원 “범죄 소명 충분하지 않고 증거인멸 우려 단정 어려워”
[Crawl RSS] http://rss.etnews.com/Section904.xml
Duplicated Title: 폴스타, 폴스타3 사막 주행 테스트 완료…내년 1분기 韓 출시
Duplicated Title: 우크라가 폭사했다던 러 해군 사령관…다음날 화상회의 등장
Duplicated Title: 두께 7.6mm...모토로라, 5G 스마트폰 '엣지 40' 국내 출시
Duplicated Title: 부산시 국가 R&D 예산 증액 분야에 전략 대응
Duplicated Title: “첨단기술과 디자인 하모니”…제네시스, GV80 부분변경·GV80 쿠페 최초 공개
Duplicated Title: 이재명, 구속영장 기각… 법원 “범죄 소명 충분하지 않고 증거인멸 우려 단정 어려워”
Duplicated Title: 아이폰 'No'... 미군이 갤럭시 S23을 왜?
Duplicated Titl

In [15]:
# 단계2 : list에 존재하는 모든 링크를 돌아다니며 본문 text를 추출

# 아래 함수는 하나의 url을 입력받아서, 링크를 타고 들어가, 그 안에 title과 text를 추출한다.
# 디폴트로 한글로 지정한다.
def crawl_article(url, language='ko'):
    print("[Crawl Article]",url) # 현재 title과 text를 추출한 url을 프린트
    a = Article(url, language=language) # Article을 사용하여 그 URL입력하고, 언어옵션지정, a에 저장
    a.download() # a에 해당하는 url기사 다운로드
    a.parse() # a에 해당하는 url 기사 분석
    return a.title, preprocessing(a.text) # a에 해당하는 title과 본문 출력

def preprocessing(text):
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그 제거
    return text_article



for article in list_articles: # list에 있는 모든 기사를 하나씩 방문
    _, text = crawl_article(article['link']) # 그 기사의 link를 crawl_article함수에 넣어 본문 추출
    article['text'] = text # 추출된 본문을 list_articles에 'text'라는 속성 새로 만들어 저장
print(list_articles[0]) # 첫번째 기사를 출력(title, link, text가 모두 나오는 것 확인)

[Crawl Article] https://www.etnews.com/20230927000056
[Crawl Article] https://www.etnews.com/20230927000047
[Crawl Article] https://www.etnews.com/20230927000039
[Crawl Article] https://www.etnews.com/20230927000013
[Crawl Article] https://www.etnews.com/20230926000368
[Crawl Article] https://www.etnews.com/20230927000007
[Crawl Article] https://www.etnews.com/20230926000363
[Crawl Article] https://www.etnews.com/20230926000316
[Crawl Article] https://www.etnews.com/20230926000189
[Crawl Article] https://www.etnews.com/20230926000364
[Crawl Article] https://www.etnews.com/20230926000344
[Crawl Article] https://www.etnews.com/20230926000205
[Crawl Article] https://www.etnews.com/20230926000340
[Crawl Article] https://www.etnews.com/20230926000328
[Crawl Article] https://www.etnews.com/20230926000101
[Crawl Article] https://www.etnews.com/20230924000164
[Crawl Article] https://www.etnews.com/20230926000288
[Crawl Article] https://www.etnews.com/20230926000275
[Crawl Article] https://www.

In [16]:
from abc import get_cache_token
# 단계3 : 모든 본문text에서 명사(키워드, 빈도수) 추출
def get_keywords(text, nkeywords=10): # 키워드 추출 함수, 디폴트로 10개 지정
    spliter = Okt() # konlpy에 의해서 문장을 형태소별로 쪼개는 기능을 위해 spliter 생성
    nouns = spliter.nouns(text) # spliter 에 의해서 nouns함수를 불러 text를 넣으면 그 text의 명사만 추출
    count = Counter(nouns) # 추출된 명사들의 출현빈도 추출
    list_keywords = [] # 비어있는 키워드 리스트를 먼저 만듦
    for n, c in count.most_common(nkeywords): # 가장 출현빈도 높은 명사부터 순차적으로 10번 출력
        item = {'keyword':n, 'count':c} # 리스트에 저장은 {'keyword', 'count'} 형식으로 저장됨
        list_keywords.append(item) # 이런 저장된 포맷으로 이를 list_keywords에 붙여 나감
    return list_keywords

for article in list_articles: # 모든 기사를 돌아다니면 text에서 명사추출, 키워드/빈도 추출
    keywords = get_keywords(article['text']) # get_keywords 함수로 키워드/빈도 추출
    article['keywords']=keywords # 추출된 키워드/빈도를 list_articles의 'keywords'로 저장
print(list_articles[0]) # 첫번째 기사 출력(title, link, text, keywords)

{'title': '폴스타, 폴스타3 사막 주행 테스트 완료…내년 1분기 韓 출시', 'link': 'https://www.etnews.com/20230927000056', 'text': "스웨덴 전기차 브랜드 폴스타가 아랍에미리트(UAE)에서 2주간 진행한 '폴스타3' 혹서기 테스트를 성공적으로 완료했다고 27일 밝혔다.\n\n폴스타3는 브랜드 최초의 전기 스포츠유틸리티차량(SUV)으로 내년 1분기 국내 출시된다.\n\n\n\n폴스타3 프로토타입은 지난 7월 영국 최대 자동차 축제 '영국 굿우드 페스티벌 오브 스피드'에서 처음 공개됐다. 가장 더운 환경에서 주행과 공조시스템 등을 미세 조정하기 위해 엔지니어 개발팀과 함께 지난달 UAE를 방문했다.\n\n\n\n이번 테스트는 폴스타 개발 과정에 진행되는 극한 기후 테스트 중 하나로 50도까지 기온이 올라가는 두바이와 아부다비 대도시, 사막 지역에서 진행됐다.\n\n\n\n폴스타3는 순수 전기차 디자인과 강력한 성능, 고급 편의 사양과 첨단 주행보조시스템 등 폴스타 역량과 기술을 집약한 플래그십 모델이다. 111kWh 리튬이온 배터리를 탑재해 1회 충전 시 최대 610㎞ 주행거리와 최대 517마력을 발휘한다.\n\n\n\n폴스타는 내년 1분기 폴스타3 생산을 시작해 전기 쿠페 폴스타까지 출시해 총 3가지 모델 라인업을 갖출 계획이다.\n\n토마스 잉엔라트 폴스타 최고경영자(CEO)는 “폴스타3는 여정의 시작 단계에 있으며 한국 고객도 내년 1분기에 폴스타 스페이스를 방문해 뛰어난 디자인과 독보적이고 혁신적인 인테리어를 경험할 수 있을 것”이라고 말했다.\n\n김지웅 기자 jw0316@etnews.com", 'keywords': [{'keyword': '폴', 'count': 13}, {'keyword': '스타', 'count': 13}, {'keyword': '진행', 'count': 3}, {'keyword': '테스트', 'count': 3}, {'keyword': '내년', 'count': 3}, {'k

In [ ]:
# 단계4 : 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력
query = input()

def search_articles(query, list_keywords): # 쿼리가 키워드리스트에 있으면 회수를 출력(없으면 0)
    nWords = 0 # 아래의 if문에 걸리지 않으면(즉 쿼리가 키워드에 없으면) 0을 출력하기 위함
    for kw in list_keywords: # 키워드 리스트 검색
        if eq(query, kw['keyword']): # 쿼리와 동일한 키워드가 존재한다면 ## query == kw['keyword'] 도 가능
            nWords = kw['count'] # 그러면 그 키워드에 해당하는 count가 nWords값이 됨
    return nWords # 결과적으로 쿼리의 출현횟수를 출력

for article in list_articles: # 모든 기사를 돌아다니며 입력받은 쿼리가 몇 번 나오나 검색
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0: # 쿼리가 키워드에 존재하지 않는 경우는 출력하지 않음
        print('[TF]',nQuery, '[title]',article['title'], '[URL]', article['link'])

반도체
[TF] 20 [title] 전자신문 테크서밋 내달 18일 개막…반도체·디스플레이 미래기술 엿본다 [URL] https://www.etnews.com/20230926000282
[TF] 10 [title] [에듀플러스]제2회 도쿄일렉트론코리아 드림업 반도체 캠프, 10월 21일·11월 25일 개최…무료 참가 신청은 이티에듀 홈페이지 [URL] https://www.etnews.com/20230925000316
[TF] 25 [title] "中서 현상 유지만 하라는 것…첨단 반도체 대규모 투자 어려워" [URL] https://www.etnews.com/20230924000040
[TF] 8 [title] 美, '반도체법' 가드레일 규정 최종 발표 [URL] https://www.etnews.com/20230923000006
[TF] 6 [title] 펨트론, 웨이퍼 표면 검사 장비 국산화 [URL] https://www.etnews.com/20230925000191
[TF] 10 [title] “美 반도체 보조금 받으면 중국 내 생산능력 5%만 확장 가능” [URL] https://www.etnews.com/20230922000447
